In [ ]:
import cudf

In [ ]:
test=cudf.read_csv('../input/house-prices-data/test.csv')
df=cudf.read_csv('../input/house-prices-data/train.csv')
df

In [ ]:
df.drop('Alley',axis='columns', inplace=True)
df.drop('PoolQC',axis='columns', inplace=True)
df.drop('Fence',axis='columns', inplace=True)
df.drop('MiscFeature',axis='columns', inplace=True)
test.drop('Alley',axis='columns', inplace=True)
test.drop('PoolQC',axis='columns', inplace=True)
test.drop('Fence',axis='columns', inplace=True)
test.drop('MiscFeature',axis='columns', inplace=True)

In [ ]:
import cupy
import cuml

In [ ]:
for c in df:
    df[c].fillna(df[c].mode()[0],inplace=True)
for c in test:
    test[c].fillna(test[c].mode()[0],inplace=True)

In [ ]:
from cuml.preprocessing import LabelEncoder
lb=LabelEncoder()
obj_c=list(df.select_dtypes(['object']).columns)
for c in obj_c:
    df[c]=lb.fit_transform(df[c])
    
obj_c_test=list(test.select_dtypes(['object']).columns)
for c in obj_c_test:
    test[c]=lb.fit_transform(test[c])


In [ ]:
X=df.iloc[:,1:75]
Y=df.iloc[:,76]
Y=cupy.array(Y)
Y=Y.reshape(-1,1)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X.as_matrix())


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [ ]:
from cuml import LinearRegression

In [ ]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,y_train)
    preds = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('float64'),preds))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test.astype('float64'),preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('float64'),preds))

In [ ]:
t=test.iloc[:,1:75]

In [ ]:
# t=test.iloc[:,1:76]
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
test_scaled=scaler.fit_transform(t.as_matrix())

In [ ]:
lrtest = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd')
reg = lrtest.fit(X_train,y_train)
final_preds = lr.predict(test_scaled)
test

In [ ]:
df_ans=cudf.DataFrame()
df_ans['Id']=test['Id']
df_ans['SalePrice']=final_preds
df_ans

In [ ]:
filename='./submission.csv'
df_ans.to_csv(filename,index=False)